In [5]:
# Projekt(1): Kombination of KI, Biosignal & Robotik
# Goal of Projekt: Connecting the output of the model to the prosthetic hand simulation

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report


# Synthetic EMG data generation (3 classes) and feature extraction
rng=np.random.RandomState(42)

fs=200     #Sampling rate
T=1        #Time of each sample (seconds)
n=300      #Number of signals
L=fs * T   #Length of each signal (number of points)

x,y=[],[]

for i in range(n):
  if i < n//3:              # Class (0): Without movement (rest)
     sig=rng.normal(0,0.1,L); label=0


  elif i< 2 * n//3:         # Class (1): Closed hand (fist)
       sig=rng.normal(0,0.4,L); label=1

  else:                     # Class (2): Open hand
       sig=rng.normal(0.2,0.2,L); label=2


  mean_val=sig.mean()
  rms_val=np.sqrt((sig**2).mean())
  var_val=sig.var()

  x.append([mean_val, rms_val,var_val])
  y.append(label)

x=pd.DataFrame(x,columns=["Mean","RMS","Var"])
y=np.array(y)

# Data splitting into Train/Test (with class balance preservation)
xtr,xte,ytr,yte=train_test_split(x,y,test_size=0.25, stratify=y, random_state=42)

# Model training (Random Forest)
model=RandomForestClassifier(n_estimators=200,random_state=42)
model.fit(xtr,ytr)

# Model evaluation
yp=model.predict(xte)

print("Accuracy:", round(accuracy_score(yte,yp),3))
print("Confusion Matrix:\n", confusion_matrix(yte,yp))
print(classification_report(yte,yp,digits=3))

# Connecting the model output to a simple prosthetic hand simulation (open/rest/fist)
kp=0.4  # Simple proportional control gain
angle=30.0
targets,angles=[],[]

cmds=yp.tolist()

for c in cmds:
    if c==0:
      target=angle
    elif c==1:
      target=90.0
    elif c==2:
      target=0.0

    u = kp * (target - angle)
    angle += u
    angle=np.clip(angle,0,90)

    angles.append(angle)
    targets.append(target)

    print("Sample commands:", cmds[:15])
    print("Sample angles:", [round(a,1) for a in angles[:15]])




Accuracy: 1.0
Confusion Matrix:
 [[25  0  0]
 [ 0 25  0]
 [ 0  0 25]]
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        25
           1      1.000     1.000     1.000        25
           2      1.000     1.000     1.000        25

    accuracy                          1.000        75
   macro avg      1.000     1.000     1.000        75
weighted avg      1.000     1.000     1.000        75

Sample commands: [1, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 2, 2, 1, 2]
Sample angles: [np.float64(54.0)]
Sample commands: [1, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 2, 2, 1, 2]
Sample angles: [np.float64(54.0), np.float64(54.0)]
Sample commands: [1, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 2, 2, 1, 2]
Sample angles: [np.float64(54.0), np.float64(54.0), np.float64(54.0)]
Sample commands: [1, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 2, 2, 1, 2]
Sample angles: [np.float64(54.0), np.float64(54.0), np.float64(54.0), np.float64(54.0)]
Sample commands: [1, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 2,